# Description

The goal of this code is to test my intuition on how to filter, arrange, and merge data.

In [6]:
# importing thee necessary libraries
import numpy as np
import scipy
import pandas as pd
import requests, zipfile, io
import urllib

## Download and Load the World Development Indicators data set

I will be working with the World Development Indicators data set. <br/> 
I will download this data set from the world bank databank.<br/>
Hence, my very first step will be to download the data to my computer

In [7]:
#note this took several minutes. I think it depends on the speed of the internet connection
r = requests.get('http://databank.worldbank.org/data/download/WDI_csv.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

# let us free the variales we used above
del z
del r

The above code downloaded a zip archive to my working folder, which by default was the the location of this notebook in my computer. <br/>
Secondly, and since the document downloaded is a zip archive, it extracts the documents from the archive. <br/> 
The contents include multiple .csv files, however i will be working only with the document 'WDIData.csv'. <br/>

In the cell below, we make use of Pandas to open the file "WDIData.csv" and save it to a variable called 'wdi'.<br/>

In [8]:
wdi = pd.read_csv("WDIData.csv", encoding="ISO-8859-1")

## Download and Load the Penn World Table V9.0

I will additionally be using data from the pwt v9.0 tables. <br/> 

In [9]:
# downloading dataset using the library urllib
urllib.request.urlretrieve("https://www.rug.nl/ggdc/docs/pwt90.xlsx", "pwt90.xlsx")

('pwt90.xlsx', <http.client.HTTPMessage at 0x11ccc9dd8>)

In [10]:
# opening and reading the filee 'pwt90.xlsx'
#using the read_excel pandas method for reading excel files
pwt = pd.read_excel("pwt90.xlsx", sheet_name="Data")

## Data Wrangling

Now that i have loaded my data into variable 'wdi', i am ready to start playing with it. <br/>
I will Start by printing all column values in the cell bellow.

In [11]:
print(wdi.columns)

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', 'Unnamed: 62'],
      dtype='object')


Next, i will list the values in the column 'Country Name'.<br/>
I got a list with repeated values, i will be deleting all duplicates to ease my analysis. <br/>

In [12]:
wdi['Country Name']
wdi.drop_duplicates(['Country Name'])

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Arab World,ARB,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1600,Caribbean small states,CSS,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3200,Central Europe and the Baltics,CEB,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4800,Early-demographic dividend,EAR,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6400,East Asia & Pacific,EAS,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8000,East Asia & Pacific (excluding high income),EAP,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9600,East Asia & Pacific (IDA & IBRD countries),TEA,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11200,Euro area,EMU,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12800,Europe & Central Asia,ECS,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14400,Europe & Central Asia (excluding high income),ECA,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I noticed that while the bottom rows represent Countries, the top rows represent aggregates of countries (e.g., world regions). <br/> However i am only interested in working with country-level data, and as such i need to filter out all unecessary rows.

In [13]:
cnames = wdi['Country Name']
cnames = cnames.drop_duplicates()

In [14]:
cnames[0:51]

0                                               Arab World
1600                                Caribbean small states
3200                        Central Europe and the Baltics
4800                            Early-demographic dividend
6400                                   East Asia & Pacific
8000           East Asia & Pacific (excluding high income)
9600            East Asia & Pacific (IDA & IBRD countries)
11200                                            Euro area
12800                                Europe & Central Asia
14400        Europe & Central Asia (excluding high income)
16000         Europe & Central Asia (IDA & IBRD countries)
17600                                       European Union
19200             Fragile and conflict affected situations
20800               Heavily indebted poor countries (HIPC)
22400                                          High income
24000                                            IBRD only
25600                                     IDA & IBRD tot

I verified that the first 48 values in cnames 'Country Name' do not correspond to countries, but aggregates.<br/>
In the next cell, i will filter out, from 'wdi', rows in which 'Country Name' represents an aggregate of countries.<br/>

In [15]:
wdi = wdi.loc[~wdi['Country Name'].isin(cnames[0:47])]

In [16]:
# resetting thee indexes of wdi, performing this operation in places
wdi.reset_index(inplace = True)

In [17]:
wdi

,index,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,75200,Afghanistan,AFG,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,75201,Afghanistan,AFG,"2005 PPP conversion factor, private consumptio...",PA.NUS.PRVT.PP.05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,75202,Afghanistan,AFG,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,...,1.884000e+01,2.068000e+01,2.233000e+01,2.408000e+01,2.617000e+01,2.799000e+01,3.010000e+01,3.244000e+01,NaN,NaN
3,75203,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,...,4.485489e+01,4.270000e+01,4.322202e+01,6.910000e+01,6.725955e+01,8.950000e+01,7.150000e+01,8.413714e+01,NaN,NaN
4,75204,Afghanistan,AFG,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,...,3.184516e+01,3.240000e+01,3.338011e+01,6.380000e+01,5.842367e+01,8.780000e+01,6.420000e+01,7.896107e+01,NaN,NaN
5,75205,Afghanistan,AFG,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,...,8.535451e+01,8.280000e+01,8.656778e+01,9.500000e+01,9.257729e+01,9.870000e+01,9.250000e+01,9.803841e+01,NaN,NaN
6,75206,Afghanistan,AFG,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.005013e+00,NaN,NaN,9.961000e+00,NaN,NaN,1.489331e+01,NaN
7,75207,Afghanistan,AFG,Account ownership at a financial institution o...,FX.OWN.TOTL.FE.ZS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.616230e+00,NaN,NaN,3.812426e+00,NaN,NaN,7.160685e+00,NaN
8,75208,Afghanistan,AFG,Account ownership at a financial institution o...,FX.OWN.TOTL.MA.ZS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.541546e+01,NaN,NaN,1.578467e+01,NaN,NaN,2.253650e+01,NaN
9,75209,Afghanistan,AFG,Account ownership at a financial institution o...,FX.OWN.TOTL.OL.ZS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.053888e+01,NaN,NaN,1.150844e+01,NaN,NaN,1.801650e+01,NaN


## Indicator Codes and Indicator Name

I selected the columns 'Indicator Name' and 'Indicator Code'.<br/> 
I deleted all duplicates, and then printed the top 5 and bottom 5 values. <br/>

In [18]:
wdi[['Indicator Name','Indicator Code']].drop_duplicates().head()

,Indicator Name,Indicator Code
0,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05
1,"2005 PPP conversion factor, private consumptio...",PA.NUS.PRVT.PP.05
2,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS
3,Access to electricity (% of population),EG.ELC.ACCS.ZS
4,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS


In [19]:
wdi[['Indicator Name','Indicator Code']].drop_duplicates().tail()

,Indicator Name,Indicator Code
1595,Women who believe a husband is justified in be...,SG.VAW.NEGL.ZS
1596,Women who believe a husband is justified in be...,SG.VAW.REFU.ZS
1597,Women who were first married by age 15 (% of w...,SP.M15.2024.FE.ZS
1598,Women who were first married by age 18 (% of w...,SP.M18.2024.FE.ZS
1599,Women's share of population ages 15+ living wi...,SH.DYN.AIDS.FE.ZS


In [20]:
# created a new dataframe made up of columns "Indicator name and indicator code"
# deleted duplicates
# set the column 'Indicator Code' as the index of indicators
indicators = ((wdi[['Indicator Name','Indicator Code']]).drop_duplicates()).set_index(['Indicator Code'])

Using 'indicators' DataFrame, i will be finding the 'Indicator Code' associated with the following observables:
1. 'Population',
2. 'GDP', 
3. 'GINI index'

In [21]:
indicators[indicators['Indicator Name'].str.contains('Population, total')]

,Indicator Name
Indicator Code,
SP.POP.TOTL,"Population, total"


In [ ]:
pd.set_option('display.max_rows', 500)
indicators[indicators['Indicator Name'].str.contains("GDP")]

In [22]:
indicators[indicators['Indicator Name'].str.contains('GINI index')]

,Indicator Name
Indicator Code,
SI.POV.GINI,GINI index (World Bank estimate)


## Extracting and Cleaning data from WDI and PWT

From 'wdi' extract the columns 'Indicator Code', 'Country Code', and '2002'.
Save the output in variable 'wdi_sample'

In [23]:
wdi_sample = wdi[['Indicator Code', 'Country Code', '2002']]

In [24]:
# selecting the lines associated with the indicator codes from wdi_sample
print(wdi_sample.loc[wdi_sample['Indicator Code'].isin(['SI.POV.GINI','SP.POP.TOTL','NY.GDP.MKTP.CD'])])

        Indicator Code Country Code          2002
525     NY.GDP.MKTP.CD          AFG  4.128821e+09
549        SI.POV.GINI          AFG           NaN
1147       SP.POP.TOTL          AFG  2.197992e+07
2125    NY.GDP.MKTP.CD          ALB  4.435079e+09
2149       SI.POV.GINI          ALB  3.170000e+01
2747       SP.POP.TOTL          ALB  3.051010e+06
3725    NY.GDP.MKTP.CD          DZA  5.676029e+10
3749       SI.POV.GINI          DZA           NaN
4347       SP.POP.TOTL          DZA  3.199505e+07
5325    NY.GDP.MKTP.CD          ASM  5.140000e+08
5349       SI.POV.GINI          ASM           NaN
5947       SP.POP.TOTL          ASM  5.873100e+04
6925    NY.GDP.MKTP.CD          AND  1.733117e+09
6949       SI.POV.GINI          AND           NaN
7547       SP.POP.TOTL          AND  7.004900e+04
8525    NY.GDP.MKTP.CD          AGO  1.249735e+10
8549       SI.POV.GINI          AGO           NaN
9147       SP.POP.TOTL          AGO  1.757265e+07
10125   NY.GDP.MKTP.CD          ATG  8.146153e+08


In [25]:
wdi_sample = wdi_sample.loc[wdi_sample['Indicator Code'].isin(['SI.POV.GINI','SP.POP.TOTL','NY.GDP.MKTP.CD'])]

In [26]:
# creating a pivot table in which values are the column '2002', the index is the 'Country Code', and the columns are the Indicator Codes.
wdi_sample = wdi_sample.pivot_table(values='2002',index='Country Code', columns='Indicator Code')

In [27]:
# renaming the column names of wdi_sample to 'Population', 'GDP', and 'GINI'
wdi_sample.columns = ['GDP', 'GINI', 'Population']

In [28]:
# selecting from "pwt" only values of the yaer 2002
# extracting "countrycode" and "hc"
# renaming 'countrycode' to 'Country Code'
pwt_sample =  pwt.loc[pwt['year'] == 2002][['countrycode','hc']].rename(columns={'countrycode': 'Country Code'})

In [34]:
# creating a new dataframe that contains the columns from wdi_sample and pwt_sample, matched by 'Country Code'
data = pd.concat([pwt_sample.set_index('Country Code'), wdi_sample], axis = 1)

/Users/adeoluwaakande/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


Considering the data for the year 2002 that i have prepared above, i will
 be performing the necessary data manipulations to answer the following questions:

1. Which countries have a population size of 10 million habitations +/- 1 million?
2. What is the average and the standard deviation in GDP of countries listed in 1?
3. What is the average and the standard deviation in the GDP of countries NOT listed in 1?
4. Repeat point 2 and 3 but for the GDP per capita.
5. What is the Country with the highest Human Capital (hc in the PWT tables)?
6. What is the Country with the Lowest Human Capital (hc in the PWT tables)?


In [35]:
data = data.reset_index()
data.rename(columns={'index':'Country'}, inplace=True)
Q1 = data.loc[(data['Population'] >= 9000000) & (data['Population'] <= 11000000), 'Country']
print(Q1)

15     BEL
23     BLR
50     CZE
73     GIN
77     GRC
87     HUN
161    PRT
171    SEN
177    SOM
190    TCD
198    TUN
Name: Country, dtype: object


In [36]:
mean_Q1 = (data.loc[data['Country'].isin(Q1)])["GDP"].mean()
std_Q1 = (data.loc[data['Country'].isin(Q1)])["GDP"].std()
print("The average is ",mean_Q1, "and the standard deviation is ", std_Q1)

The average is  74455540556.17746 and the standard deviation is  85204567133.23154


In [37]:
mean_Q3 = (data.loc[~data['Country'].isin(Q1)])["GDP"].mean()
std_Q3 = (data.loc[~data['Country'].isin(Q1)])["GDP"].std()
print("The average is ",mean_Q3, "and the standard deviation is ", std_Q3)

The average is  173731828490.20428 and the standard deviation is  880372555108.6028


In [38]:
mean_Q4_in = ((data.loc[data['Country'].isin(Q1)])["GDP"] / (data.loc[data['Country'].isin(Q1)])["Population"]).mean()
std_Q4_in = ((data.loc[data['Country'].isin(Q1)])["GDP"] / (data.loc[data['Country'].isin(Q1)])["Population"]).std()
mean_Q4_out = ((data.loc[~data['Country'].isin(Q1)])["GDP"] / (data.loc[~data['Country'].isin(Q1)])["Population"]).mean()
std_Q4_out = ((data.loc[~data['Country'].isin(Q1)])["GDP"] / (data.loc[~data['Country'].isin(Q1)])["Population"]).std()
print(mean_Q4_in,std_Q4_in, mean_Q4_out, std_Q4_out)

7162.581644120961 8142.612572151089 8826.605448799437 14155.47950935825


In [39]:
print(data[['Country']][data.hc == data.hc.max()])

    Country
206     USA


In [40]:
print(data[['Country']][data.hc == data.hc.min()])

   Country
17     BFA
